In [95]:
import argparse
import csv
import os
import json
import sys
sys.path.append("../")

import torch
from tqdm.notebook import tqdm
import transformers
from fastDamerauLevenshtein import damerauLevenshtein

from data.dataset import TypoDataset, TypoOnlineDataset
from model.char_lm import CharacterLanguageModel, CharTokenizer, EditDistanceCrossEntropyLoss, Trie

In [96]:
args = argparse.Namespace()
args.mimic_csv_dir = "../../mimic3/noteevents_split"
# args.data_dir = "../data/mimic_synthetic_trans"
args.dict_file = "../data/new_vocab/lexicon.json"
args.bert_dir = "../../bert_models/"
# args.output_dir = ""
args.is_train = False
# args.test_file = "test.csv"
# args.init_ckpt = ""
args.init_step = 0
args.decoder_layers = 12
args.train_bert = True
args.dropout = 0.1
args.label_smoothing = 0.0
args.batch_size = 1
args.num_gpus = 1
args.do_substitution = True
args.do_transposition = True
args.max_word_corruptions = 2
args.no_corruption_prob = 0.0
# args.num_beams = 30
# args.edit_distance_weight = 5.0
args.length_penalty = 1.0
args.train_with_ed = False
args.beam_sort_linear_ed = False
args.beam_final_score_normalize_ed = True
args.dict_matching = True

In [106]:
# Files
dataset_fpath = '../data/mimic_clinspell_new/test.tsv'
# ckpt_fpath = '../../exp/cond_indep_typo/mimic_synthetic_12layers6/ckpt.pkl'
ckpt_fpath = '../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/ckpt-160000.pkl'
# ckpt_fpath = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws/ckpt-105000.pkl'

# Parameters
args.data_dir = os.path.dirname(dataset_fpath)
args.test_file = os.path.basename(dataset_fpath)
args.init_ckpt = ckpt_fpath
args.num_beams = 30
args.edit_distance_weight = 5.0

In [98]:
# BERT and LevT configs
bert_config_file = os.path.join(args.bert_dir, 'bert_config.json')
bert_config = CharacterLanguageModel.get_bert_config(bert_config_file)
char_decoder_config = CharacterLanguageModel.get_char_decoder_config(bert_config_file, args)

# BERT tokenizer and LevT tokenizer
bert_vocab_file = os.path.join(args.bert_dir, 'vocab.txt')
bert_tokenizer = CharacterLanguageModel.get_bert_tokenizer(bert_vocab_file)
typo_tokenizer = CharTokenizer()

# Trie
if args.dict_matching:
    trie = Trie(typo_tokenizer, eos_token_id=typo_tokenizer.eos_index)
    with open(args.dict_file, 'r') as fd:
        dict_words = json.load(fd)
    trie.add_words(dict_words)
else:
    trie = None

In [79]:
# Load datset
dataset_test = TypoDataset(os.path.join(args.data_dir, args.test_file), bert_tokenizer, typo_tokenizer)
dataloader_test = torch.utils.data.DataLoader(dataset_test,
                                              batch_size=args.batch_size,
                                              shuffle=False,
                                              num_workers=0,
                                              collate_fn=dataset_test.get_collate_fn())

test_examples = {}
with open(os.path.join(args.data_dir, args.test_file), 'r') as fd:
    reader = csv.reader(fd, delimiter='\t')
    for i, row in enumerate(reader):
        if i == 0: continue
        test_examples[int(row[0])] = row

Read file ../data/mimic_clinspell_new/test.tsv... 873 rows
Parsing rows (64 processes)


100%|██████████| 873/873 [00:01<00:00, 636.28it/s]


In [107]:
# BERT encoder (context) and decoder (character) model
bert_tf_file = os.path.join(args.bert_dir, 'bert_model.ckpt')
bert_model = CharacterLanguageModel.build_bert_model(bert_config)
bert_model.load_tf_weights(config=bert_config, tf_checkpoint_path=bert_tf_file)
char_embeddings_decoder = CharacterLanguageModel.get_char_embeddings_from_bert(
    bert_model.cls.predictions.decoder.weight, bert_tokenizer)
char_decoder = CharacterLanguageModel.build_char_decoder(char_decoder_config,
                                                         char_embeddings_decoder)
%recall jupyter_notebook_cell_number
# The main model
model = CharacterLanguageModel(args, char_decoder_config, bert_model, char_decoder)
if args.init_ckpt is not None:
    print(f'Load model ckpt from {args.init_ckpt} (step {args.init_step})')
    model.load_state_dict(torch.load(args.init_ckpt))
    
model.cuda()
model.eval();

Couldn't evaluate or find in history: jupyter_notebook_cell_number
Load model ckpt from ../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/ckpt-160000.pkl (step 0)


In [70]:
# Get the example
def get_examples_from_typo(typo):
    return [e for e in dataset_test if e['typo'] == typo]

def get_words(token_ids):
    """ Print words from token_ids """
    num_words = token_ids.size()[0]
    words = []
    for i in range(num_words):
        tokens = typo_tokenizer.convert_ids_to_tokens(token_ids[i,:])
        while True:
            if tokens[-1] == '<pad>':
                tokens = tokens[:-1]
            else: break
        words.append(''.join(tokens))
    return words

In [71]:
def get_beam_search_and_correct_score(example, print_beam_steps=False):
    # Beam search
    with torch.no_grad():
        context_tokens = torch.LongTensor([example['context_tokens']]).cuda()
        context_attention_mask = torch.LongTensor([example['context_attention_mask']]).cuda()
        typo_token_ids = torch.LongTensor([example['typo_token_ids']+[1]]).cuda()
        typo_attention_mask = torch.LongTensor([example['typo_attention_mask']+[0]]).cuda()
        typo_words = get_words(typo_token_ids)
        correct_token_ids = torch.LongTensor([example['correct_token_ids']+[1]]).cuda()
        correct_attention_mask = torch.LongTensor([example['correct_attention_mask']+[0]]).cuda()
        correct_words = get_words(correct_token_ids)

        decoder_token_ids, decoder_scores, lm_scores = model.generate(input_ids=context_tokens,
                                                       attention_mask=context_attention_mask,
                                                       context_attention_mask=context_attention_mask,
                                                       use_cache=False,
                                                       decoder_start_token_id=0,
                                                       bos_token_id=0, pad_token_id=1, eos_token_id=2,
                                                       max_length=64, min_length=3, num_beams=args.num_beams,
                                                       num_return_sequences=args.num_beams,
                                                       typo_token_ids=typo_token_ids,
                                                       length_penalty=args.length_penalty,
                                                       edit_distance_weight=args.edit_distance_weight,
                                                       beam_sort_linear_ed=args.beam_sort_linear_ed,
                                                       beam_final_score_normalize_ed=args.beam_final_score_normalize_ed,
                                                       trie=trie, print_beam_steps=print_beam_steps)
        decoder_words = get_words(decoder_token_ids)

        result = {
            'example_id': example['example_id'],
            'note_id': example['note_id'],
            'typo': typo_words[0],
            'correct': correct_words[0],
            'output': decoder_words,
            'scores': decoder_scores,
            'lm_scores': lm_scores,
        }

        # Get the correct score
        output_logits = model.forward(context_tokens,
                                      context_attention_mask,
                                      correct_token_ids,
                                      correct_attention_mask,
                                      return_dict=True)

        correct_label = correct_token_ids[:, 1:].unsqueeze(-1)  # Remove first BOS
        correct_label_mask = correct_attention_mask[:, 1:].float()

        lprobs = torch.nn.functional.log_softmax(output_logits[:, :-1], dim=-1)
        char_scores = lprobs.gather(dim=-1, index=correct_label).squeeze(-1)
        lm_scores = (char_scores * correct_label_mask).sum(-1)
        j = 0
        lm_score = float(lm_scores[j]) / (len(correct_words[j]) - 6) ** args.length_penalty
        ed_score = damerauLevenshtein(typo_words[j], correct_words[j], similarity=False) * args.edit_distance_weight * -1.0
        if args.beam_sort_linear_ed:
            ed_score *= len(correct_words[j]) - 6
        if args.beam_final_score_normalize_ed:
            ed_score /= (len(correct_words[j]) - 6) ** args.length_penalty
        score = lm_score + ed_score

        correct_output = {
            'example_id': example['example_id'],
            'note_id': example['note_id'],
            'typo': typo_words[j],
            'correct': correct_words[j],
            'score': score,
            'lm_score': lm_score,
            'char_scores': char_scores[j]
        }
    
    return result, correct_output

In [72]:
def textbf(text):
    return '\033[1m' + text + '\033[0m'

def textcolor(text, color):
    code = {'red': '31', 'green': '32', 'yellow': '33', 'blue': '34', 'magenta': '35', 'cyan': '36', 'white': '37'}
    return '\033[' + code[color] + 'm' + text + '\033[0m'

def print_results(example, outputs, model_name=None, correct_output=None, show_context=False, context_length=10):
    typo, left, right, correct = example[-4:]
    print(' [Input]')
    print(f'typo / correct : {typo} / {correct}')
    if show_context:
        left = ' '.join(left.split()[-context_length:])
        right = ' '.join(right.split()[:context_length])
        print(f'Context: \"{left + " " + textcolor(textbf(typo), "red") + " " + right}\"')
    
    print(' [Output]')
    output_words = [o[3:-4] for o in outputs['output']]
    rank = output_words.index(correct)+1 if correct in output_words else "X"

    if model_name is None:
        model_name = 'Output'
    if rank == 1:
        model_color = 'green'
    elif rank == 'X':
        model_color = 'red'
    else:
        model_color = 'yellow'
    print(f'{textcolor(model_name, model_color) + "(" + str(rank) + ")":<35}' + "  Score      LM      ED")

    for i in range(len(output_words)):
        o = outputs['output'][i][3:-4]
        s = outputs['scores'][i]
        l = outputs['lm_scores'][i]
        e = s - l
        if o == correct:
            o = f'{textbf(o):<33}'
        else:
            o = f'{o:<25}'
        print(o + f' {s:7.4f} {l:7.4f} {e:7.4f}')

    texts = []
    s = correct_output['score']
    l = correct_output['lm_score']
    e = s - l
    print(f'{"(Correct)":<25} {s:7.4f} {l:7.4f} {e:7.4f}')

In [73]:
def parse_beam_search_cap(cap_str):
    def parse_beam_line(line):
        '<s>potassium-los: -39.638958 / -84.638962'
        split = line.split()
        return (split[0][:-1], float(split[1]), float(split[3]))
    
    def parse_hyp_line(line):
        "(<s>post-anal): -4.042214 / -1.542214"
        split = line.split()
        return (split[0][1:-2], float(split[1]), float(split[3]))
    
    lines = cap_str.split('\n')
    output = []
    cur_beam, generated_beam = [], []
    for line in lines:
        if line.startswith('Beam search intermediate'):
            if cur_beam:
                output.append((cur_len, cur_beam, generated_beam))
            cur_len = int(line.split()[-1])     
            cur_beam, generated_beam = [], []
        elif line.startswith('<s>'):
            cur_beam.append(parse_beam_line(line))
        elif line.startswith('(<s>'):
            generated_beam.append(parse_hyp_line(line))
    output.append((cur_len, cur_beam, generated_beam))
    return output

def print_beam_search_steps(beam_search_steps, example, correct_output=None):
    correct = "<s>" + example['correct'] + "</s>"
    typo = "<s>" + example['typo'] + "</s>"
    for cur_len, cur_beam, gen_beam in beam_search_steps:
        print(f' [Length {cur_len}]')

        texts = []
        texts.append('Current Beam                  Score       LM       ED | ')
        for output, un_lm_score, un_final_score in cur_beam:
            output_text = f'{textbf(textcolor(output[3:], "red")):42s}' if correct.startswith(output) else f'{output[3:]:25s}'
            texts.append(f'{output_text} {un_final_score:9.4f}{un_lm_score:9.4f}{un_final_score-un_lm_score:9.4f} | ')
        if correct_output:
            lm_score = sum(correct_output['char_scores'][:cur_len-1])
            ed_score = -args.edit_distance_weight * damerauLevenshtein(typo[2:-3], correct[2:min(cur_len+2, len(correct)-3)], similarity=False)
#             ed_score = -args.edit_distance_weight * damerauLevenshtein(typo[2:min(cur_len+2, len(correct)-3)], correct[2:min(cur_len+2, len(correct)-3)], similarity=False)
            texts.append(f'{correct[3:min(cur_len+2, len(correct)-4)] + "(Correct)":25s} {lm_score+ed_score:9.4f}{lm_score:9.4f}{ed_score:9.4f}')
            
        texts[0] += 'Generated Beam               Score       LM       ED'
        for i, (output, score, lm_score) in enumerate(gen_beam):
            output_text = f'{textbf(textcolor(output[3:], "red")):42s}' if correct[:-4] == output else f'{output[3:]:25s}'
            texts[i+1] = texts[i+1] + f'{output_text:20s}{score:9.4f}{lm_score:9.4f}{score-lm_score:9.4f}'
        print('\n'.join(texts) + '\n')

# Demo Here!

In [102]:
outputss, correct_outputs = [], []
for example in tqdm(dataset_test):
    outputs, correct_output = get_beam_search_and_correct_score(example)
    outputss.append(outputs)
    correct_outputs.append(correct_output)

In [29]:
total_cnt = len(outputss)
correct_cnt = 0
for example, outputs, correct_output in zip(dataset_test, outputss, correct_outputs):
    if example['correct'] == outputs['output'][0][3:-4]:
        correct_cnt += 1
        
print(f'{correct_cnt}/{total_cnt} = {correct_cnt/total_cnt:.4f}')

722/873 = 0.8270


In [105]:
outputss2, correct_outputs2 = [], []
for example in tqdm(dataset_test):
    outputs, correct_output = get_beam_search_and_correct_score(example)
    outputss2.append(outputs)
    correct_outputs2.append(correct_output)

In [102]:
total_cnt = len(outputss)
correct_cnt = 0
for example, outputs, correct_output in zip(dataset_test, outputss2, correct_outputs2):
    if example['correct'] == outputs['output'][0][3:-4]:
        correct_cnt += 1
        
print(f'{correct_cnt}/{total_cnt} = {correct_cnt/total_cnt:.4f}')

713/873 = 0.8167


In [37]:
def get_accuracy_typo(typo, outputss_, correct_outputs_):
    idxs = [i for i, e in enumerate(dataset_test) if e['typo'] == typo]
    total_cnt, correct_cnt = len(idxs), 0
    for i in idxs:
        example, outputs, correct_output = dataset_test[i], outputss_[i], correct_outputs_[i]
        if example['correct'] == outputs['output'][0][3:-4]:
            correct_cnt += 1
    return f'{correct_cnt}/{total_cnt} = {correct_cnt/total_cnt:.4f}'

In [34]:
from collections import Counter
typos = list(zip(*Counter([e['typo'] for e in dataset_test]).most_common()))[0]

In [101]:
for t in typos:
    print(f'{t:20s} {get_accuracy_typo(t)} {get_accuracy_typo(t, outputss2, correct_outputs2)}')

lugns                5/5 = 1.0000 5/5 = 1.0000
responsvie           5/5 = 1.0000 5/5 = 1.0000
pylenonephritis      4/5 = 0.8000 4/5 = 0.8000
noteed               5/5 = 1.0000 5/5 = 1.0000
hypercholesteromia   0/5 = 0.0000 0/5 = 0.0000
spenomegaly          2/5 = 0.4000 4/5 = 0.8000
reporteldy           5/5 = 1.0000 5/5 = 1.0000
umcomplicated        5/5 = 1.0000 5/5 = 1.0000
arimdex              4/5 = 0.8000 2/5 = 0.4000
aroound              5/5 = 1.0000 4/5 = 0.8000
claoric              3/5 = 0.6000 4/5 = 0.8000
incontinenece        5/5 = 1.0000 5/5 = 1.0000
incrrease            5/5 = 1.0000 4/5 = 0.8000
iatrogentic          4/5 = 0.8000 4/5 = 0.8000
requirs              5/5 = 1.0000 4/5 = 0.8000
aprropriately        5/5 = 1.0000 5/5 = 1.0000
popletial            2/5 = 0.4000 3/5 = 0.6000
dicarded             5/5 = 1.0000 0/5 = 0.0000
respoding            5/5 = 1.0000 5/5 = 1.0000
pevlic               5/5 = 1.0000 5/5 = 1.0000
syndomre             5/5 = 1.0000 5/5 = 1.0000
respiraoty   

deveopling           1/1 = 1.0000 1/1 = 1.0000
resuults             1/1 = 1.0000 1/1 = 1.0000
brochodilations      0/1 = 0.0000 1/1 = 1.0000
abnormalityin        1/1 = 1.0000 1/1 = 1.0000
coolaborate          1/1 = 1.0000 1/1 = 1.0000
parafaulcine         1/1 = 1.0000 1/1 = 1.0000
consoliodation       1/1 = 1.0000 1/1 = 1.0000
atributes            0/1 = 0.0000 1/1 = 1.0000
satruation           1/1 = 1.0000 1/1 = 1.0000
incrwase             1/1 = 1.0000 1/1 = 1.0000
infeoseptal          1/1 = 1.0000 1/1 = 1.0000
resusitaton          1/1 = 1.0000 1/1 = 1.0000
dehiscene            1/1 = 1.0000 1/1 = 1.0000
fluctuatio           1/1 = 1.0000 1/1 = 1.0000
autocoidal           0/1 = 0.0000 0/1 = 0.0000
phebilitis           0/1 = 0.0000 1/1 = 1.0000
difinitely           1/1 = 1.0000 1/1 = 1.0000
requiriemnts         1/1 = 1.0000 1/1 = 1.0000
psitacci             0/1 = 0.0000 0/1 = 0.0000
imaginig             1/1 = 1.0000 1/1 = 1.0000
stabilizind          1/1 = 1.0000 1/1 = 1.0000
rehospializat

In [20]:
[i for i, e in enumerate(dataset_test) if e['typo'] == 'pylenonephritis']

[27, 28, 29, 30, 31]

In [263]:
examples = get_examples_from_typo('lugns')
example = examples[0]

In [115]:
examples = get_examples_from_typo('popletial')
example = examples[4]

In [17]:
examples = get_examples_from_typo('hypercholesteromia')
example = examples[0]

In [16]:
examples = get_examples_from_typo('isoechic')
example = examples[0]

In [37]:
examples = get_examples_from_typo('sclerosin')
example = examples[0]

In [118]:
examples = get_examples_from_typo('dicarded')
example = examples[2]

In [81]:
examples = get_examples_from_typo('hypolucency')
example = examples[0]

In [108]:
examples = get_examples_from_typo('spenomegaly')
example = examples[0]

In [119]:
%%capture cap --no-stderr
outputs, correct_output = get_beam_search_and_correct_score(example, print_beam_steps=True)

In [120]:
# nega vs sple
# spenomegaly

# ED(spenomegaly, nega) = 8
# ED(spenomegaly, sple) = 9

# ED(spenom, nega) = 5
# ED(spenom, sple) = 4

In [121]:
print_results(test_examples[example['example_id']], outputs, correct_output=correct_output,
              show_context=True, context_length=10)

 [Input]
typo / correct : dicarded / discarded
Context: "abdomen is soft bs audible d-stick bright yellow green aspirates dicarded voiding qs no stool thus far cont to monitor"
 [Output]
Output(X)                   Score      LM      ED
decreased                 -2.5007 -0.5007 -2.0000
disregarded               -2.9425 -1.2758 -1.6667
diarrhea                  -2.9941 -0.7718 -2.2222
directed                  -3.1475 -0.9252 -2.2222
diagnosed                 -3.2159 -0.7159 -2.5000
cardioverted              -3.4255 -0.7332 -2.6923
carded                    -3.5269 -2.0983 -1.4286
decrease                  -3.6601 -0.8823 -2.7778
decreases                 -3.6805 -1.1805 -2.5000
addressed                 -3.7170 -0.7170 -3.0000
diameter                  -3.7338 -0.9560 -2.7778
diapered                  -3.7928 -1.5706 -2.2222
decreasing                -3.8346 -0.6528 -3.1818
diabetes                  -3.8847 -1.1069 -2.7778
disregarding              -3.9435 -1.2512 -2.6923
diarrheal    

In [122]:
beam_search_steps = parse_beam_search_cap(cap.stdout)
print_beam_search_steps(beam_search_steps, example, correct_output)

 [Length 2]
Current Beam                  Score       LM       ED | Generated Beam               Score       LM       ED
a                          -42.3267  -2.3267 -40.0000 | 
c                          -42.4283  -2.4283 -40.0000 | 
d                          -42.6984  -2.6984 -40.0000 | 
r                          -43.0441  -3.0441 -40.0000 | 
i                          -43.3635  -3.3635 -40.0000 | 
e                          -43.6749  -3.6749 -40.0000 | 
s                          -47.4163  -2.4163 -45.0000 | 
t                          -47.4538  -2.4538 -45.0000 | 
p                          -47.7126  -2.7126 -45.0000 | 
w                          -47.7830  -2.7830 -45.0000 | 
n                          -47.9298  -2.9298 -45.0000 | 
b                          -48.0065  -3.0065 -45.0000 | 
h                          -48.0753  -3.0753 -45.0000 | 
f                          -48.2347  -3.2347 -45.0000 | 
m                          -48.2571  -3.2571 -45.0000 | 
l                       